# A Few Real World Examples

A good boilerplate template.

In [1]:
def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

## Timing Functions

In [2]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function."""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f'Finished {func.__name__!r} in {run_time:.4f} seconds.')
        return value
    return wrapper_timer

In [3]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum(number ** 2 for number in range(10_000))

In [4]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0004 seconds.


In [5]:
waste_some_time(999)

Finished 'waste_some_time' in 0.4471 seconds.


## Debugging Code

The following @trace decorator will print a function's arguments and
return value every time you call the function.

In [6]:
def trace(func):
    @functools.wraps(func)
    def wrapper_trace(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f'{k}={repr(v)}' for k, v in kwargs.items()]
        # Consider `inspect.signature(func)`
        signature = ', '.join(args_repr + kwargs_repr)
        print(f'Calling {func.__name__}({signature})')
        value = func(*args, **kwargs)
        print(f'{func.__name__} returned {repr(value)}')
        return value
    return wrapper_trace

In [7]:
@trace
def make_greeting(name, age=None):
    if age is None:
        return f'Howdy, {name}!'
    else:
        return f"Whoa, {name}! {age} already, you're growing up!"

In [8]:
make_greeting('Benjamin')

Calling make_greeting('Benjamin')
make_greeting returned 'Howdy, Benjamin!'


'Howdy, Benjamin!'

In [9]:
make_greeting('Juan', age=114)

Calling make_greeting('Juan', age=114)
make_greeting returned "Whoa, Juan! 114 already, you're growing up!"


"Whoa, Juan! 114 already, you're growing up!"

In [10]:
make_greeting('Maria', age=116)

Calling make_greeting('Maria', age=116)
make_greeting returned "Whoa, Maria! 116 already, you're growing up!"


"Whoa, Maria! 116 already, you're growing up!"

"This example might not seem immediately useful since the `@trace`
decorator just repeats what you wrote. It's more powerful when
applied to small convenience functions that you **don't** call
**directly yourself.**"

In [11]:
import math

math.factorial = trace(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In this example, you apply a decorator to a function, `factorial()`,
defined in the standard library. One cannot use the `pie` syntax;
that is, `@trace`; however, one **can** wrap `math.factoria()`
inside `trace()`.

In [12]:
approximate_e(terms=5)

Calling factorial(0)
factorial returned 1
Calling factorial(1)
factorial returned 1
Calling factorial(2)
factorial returned 2
Calling factorial(3)
factorial returned 6
Calling factorial(4)
factorial returned 24


2.7083333333333335

## Slowing Down Code

In this section, we create a decorator that slows down our code.

But why would one want to do that?

Probably the most common case is that one wants to rate-limit a
function that continually checks whether a resource, like a web page,
has changed.

In [13]:
import time
def slow_down(func):
    """Sleep one second before calling the wrapped function."""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

In [14]:
@slow_down
def countdown(from_number):
    if from_number < 1:
        print('Liftoff!')
    else:
        print(from_number)
        countdown(from_number - 1)

In [15]:
countdown(3)

3
2
1
Liftoff!


## Registering Plugins

Decorators don't actually need to wrap the function that they decorate.
Instead, they can simply register that a function exists and return it
**unwrapped**. You can use this, for example, to create a lightweight
plugin architecture.

In [16]:
PLUGINS = dict()

def register(func):
    """Register a function as a plugin."""
    PLUGINS[func.__name__] = func
    return func

Note that you **do not** need to write an inner function in this
scenario because you are simply returning the original function
**unwrapped**.

We can now register functions as follows.

In [17]:
@register
def say_hello(name):
    return f'Hello, {name}!'

@register
def be_awesome(name):
    return f"Yo {name}, together we're the awesoest!"

Notice that the `PLUGINS` dictionary **already** contains references to the
two functions that have (already) been registered.

In [18]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

After registering these functions, you can use `PLUGINS` to **call**
these registered functions.

In [19]:
import random

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f'Using {greeter}')
    return greeter_func(name)

randomly_greet('Alice')

Using be_awesome


"Yo Alice, together we're the awesoest!"

In [20]:
randomly_greet('Alice')

Using be_awesome


"Yo Alice, together we're the awesoest!"

## Authenticating Users

Our final example, before moving on to some fancier decorators, is
commonly used when working with a web framework. We'll use `Flask`
to set up a web page at the URL `/secret' that is only visible to
users which are **already logged in** (or authenticated).

In [21]:
from flask import Flask, g, request, redirect, url_for

app = Flask(__name__)

def login_required(func):
    """Make sure that the user is logged in **before** proceeding.."""
    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for('login', next=request.url))
        return func(*args, **kwargs)
    return wrapper_login_required

@app.route('/secret')
@login_required
def secret_page():
    pass # Put implementation here

Although one can write these types of decorators, you should
**not usually**. Check the package for the functionality that
you need. The package functionality will typical be more
secure and still provide more functionality.